## flwの一部を使用して、opencvで顔認識を行う「自分自身」もモデルを作成する。

1.  学習データを作成する
2.  モデルを学習する　　creatstemples  -> haartrainig -> カスケードファイルが作成される
3.  モデル使って評価する

## 学習データの作成方法

学習データのフォーマット

 画像ファイル名（相対パス） 学習させたいオブジェクトの個数　 x1 y1 w1 h1......

In [30]:
# ディレクトリ内の画像データのファイル名を読み取る
# ディレクトリ構造は既知とする
import os
import cv2


BASE_DIR      = 'train_data'
JPG           = '.jpg'
CASCADE_FILE  = 'haarcascade_frontalface_default.xml'
SCALE_FACTOR  = 1.1 # 顔検出のスケールファクタ

cascade = cv2.CascadeClassifier(CASCADE_FILE)

def detect_face_object(img) :
    # 顔検出
    _img     = cv2.imread(img)
    _gray    = cv2.cvtColor(_img, cv2.COLOR_BGR2GRAY)
    _faces   = cascade.detectMultiScale(_gray, scaleFactor=SCALE_FACTOR)
    return _faces

dirs = os.listdir(BASE_DIR)
print(dirs)
for dir in dirs :
    if not os.path.isfile(dir) :
        print(" これはディレクトリです", dir)
        fnames = os.listdir(os.path.join(BASE_DIR, dir))
        for image_name in fnames :
            image_file_path = os.path.join(BASE_DIR, dir, image_name)
            if os.path.isfile(image_file_path) :
                ext = os.path.splitext(image_file_path)[1] # これで、image_fileの拡張子を取得できる
                if ext == JPG :
                    print(image_file_path, (detect_face_object(image_file_path)).shape)


['powell', 'eastwood', 'zico', '.DS_Store', 'crowford', 'vardalos', 'koizumi', 'beyonce', 'w_bush', 'rumsfeld', 'kidman']
 これはディレクトリです powell
train_data/powell/Colin_Powell_0195.jpg (2, 4)
train_data/powell/Colin_Powell_0181.jpg (1, 4)
train_data/powell/Colin_Powell_0156.jpg (1, 4)
train_data/powell/Colin_Powell_0142.jpg (1, 4)
train_data/powell/Colin_Powell_0022.jpg (1, 4)
train_data/powell/Colin_Powell_0036.jpg (2, 4)
train_data/powell/Colin_Powell_0208.jpg (1, 4)
train_data/powell/Colin_Powell_0220.jpg (2, 4)
train_data/powell/Colin_Powell_0234.jpg (1, 4)
train_data/powell/Colin_Powell_0235.jpg (1, 4)
train_data/powell/Colin_Powell_0221.jpg (1, 4)
train_data/powell/Colin_Powell_0209.jpg (1, 4)
train_data/powell/Colin_Powell_0037.jpg (1, 4)
train_data/powell/Colin_Powell_0023.jpg (1, 4)
train_data/powell/Colin_Powell_0143.jpg (2, 4)
train_data/powell/Colin_Powell_0157.jpg (2, 4)
train_data/powell/Colin_Powell_0180.jpg (1, 4)
train_data/powell/Colin_Powell_0194.jpg (2, 4)
train_data/p

AttributeError: 'tuple' object has no attribute 'shape'